In [ ]:
!pip install langchain langchain-community langchain-core
!pip install tiktoken sentence_transformers pypdf

In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향.pdf")
pages = loader.load_and_split()

In [ ]:
import tiktoken

tokenizer = tiktoken.get_encoding('cl100k_base')

def tiktoken_len(text):
  tokens = tokenizer.encode(text)
  return len(tokens)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=tiktoken_len,
)

docs = text_splitter.split_documents(pages)

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = 'jhgan/ko-sbert-nli'
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
ko = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [ ]:
!pip install ChromaDB

In [ ]:
from langchain.vectorstores import Chroma

db = Chroma.from_documents(docs, ko)

In [ ]:
query = '장기저탄소발전전략'

docs = db.similarity_search(query)
print(docs[0].page_content)

05이노비즈 정책브리프
장기저탄소발전전략 (LEDS)
∙우리나라는 녹색성장위원회 심의와 국무회의 의결을 거쳐 지난해 말 LEDS 를 UN에 제출하였으며 
부문별 전략은 다음과 같음
- (에너지공급 ) 화석연료 발전 중심의 전력공급 체계를 재생에너지와 그린수소 중심으로 전환하고 이산화탄
소포집 기술 등을 적극적으로 활용하여 전력부문의 탄소중립을 달성
- (산업) 미래 신기술 , 에너지효율 향상, 순환경제 실현으로 지속가능한 탄소중립 산업 생태계를 구축
- (수송) 청정 에너지원 (전기·수소)을 동력으로 하는 수송수단 (자동차 , 철도, 항공기 , 선박)을 확대하고 디지
털 기술을 활용한 자율주행차 , 교통 수요관리를 통해 수송부문 탄소중립 기반을 조성
- (건물) 단열과 기밀성능을 강화하고 에너지고효율 제품 사용을 확대하여 건물에서 사용되는 에너지를 최


In [ ]:
db2 = Chroma.from_documents(docs, ko, persist_directory='./chroma_db')

In [ ]:
db3 = Chroma(persist_directory='./chroma_db', embedding_function=ko)

In [ ]:
docs = db3.similarity_search(query)
print(docs[0].page_content)

05이노비즈 정책브리프
장기저탄소발전전략 (LEDS)
∙우리나라는 녹색성장위원회 심의와 국무회의 의결을 거쳐 지난해 말 LEDS 를 UN에 제출하였으며 
부문별 전략은 다음과 같음
- (에너지공급 ) 화석연료 발전 중심의 전력공급 체계를 재생에너지와 그린수소 중심으로 전환하고 이산화탄
소포집 기술 등을 적극적으로 활용하여 전력부문의 탄소중립을 달성
- (산업) 미래 신기술 , 에너지효율 향상, 순환경제 실현으로 지속가능한 탄소중립 산업 생태계를 구축
- (수송) 청정 에너지원 (전기·수소)을 동력으로 하는 수송수단 (자동차 , 철도, 항공기 , 선박)을 확대하고 디지
털 기술을 활용한 자율주행차 , 교통 수요관리를 통해 수송부문 탄소중립 기반을 조성
- (건물) 단열과 기밀성능을 강화하고 에너지고효율 제품 사용을 확대하여 건물에서 사용되는 에너지를 최


In [ ]:
docs = text_splitter.split_documents(pages)

In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 1.9 MB/s eta 0:00:00


In [ ]:
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, ko)

In [ ]:
query = '삼성전자의 동향'
results = db.similarity_search(query)

print(results[0].page_content)

07이노비즈 정책브리프
04 국내 대기업 동향 3)
∙SK그룹
- ‘20년 11월 국내 최초로 RE100 프로젝트에 가입
- 2025년까지 이천·청주 사업장의 1톤 차량을 전기 자동차로 교체할 예정
- SK하이닉스는 공정가스 제거시스템을 적용하는 사업장을 확대함
∙ 삼성전자
- 2019년 온실가스 배출량을 ‘18년 대비 8.9%, ‘08년 대비 70% 감축
- 수원사업장과 기흥사업장에 각각 1.9매가와트 , 1.5메가와트 규모의 태양광발전 설비 설치
- 제품 사용단계에서의 고효율 제품 및 저전력 반도체 개발을 통해 전력 효율화 추진
∙ LG그룹
- 15년째 세계 최대 지구촌 전등 끄기 캠페인인 ‘어스아워 ’에 동참
- 이사회 산하에 ESG 위원회 신설
- LG전자는 2030년까지 탄소중립을 달성하겠다는 목표를 수립하고 , 포스코와 협력하여 해수열냉난방시스
템, 건물일체형태양광 등 신재생에너지 관련 기술 개발에 적극 투자
∙ 현대제철


In [ ]:
result_scores = db.similarity_search_with_score(query)
print(result_scores)

[(Document(metadata={'source': '/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향.pdf', 'page': 9}, page_content='07이노비즈 정책브리프\n04 국내 대기업 동향 3)\n∙SK그룹\n- ‘20년 11월 국내 최초로 RE100 프로젝트에 가입\n- 2025년까지 이천·청주 사업장의 1톤 차량을 전기 자동차로 교체할 예정\n- SK하이닉스는 공정가스 제거시스템을 적용하는 사업장을 확대함\n∙ 삼성전자\n- 2019년 온실가스 배출량을 ‘18년 대비 8.9%, ‘08년 대비 70% 감축\n- 수원사업장과 기흥사업장에 각각 1.9매가와트 , 1.5메가와트 규모의 태양광발전 설비 설치\n- 제품 사용단계에서의 고효율 제품 및 저전력 반도체 개발을 통해 전력 효율화 추진\n∙ LG그룹\n- 15년째 세계 최대 지구촌 전등 끄기 캠페인인 ‘어스아워 ’에 동참\n- 이사회 산하에 ESG 위원회 신설\n- LG전자는 2030년까지 탄소중립을 달성하겠다는 목표를 수립하고 , 포스코와 협력하여 해수열냉난방시스\n템, 건물일체형태양광 등 신재생에너지 관련 기술 개발에 적극 투자\n∙ 현대제철'), 0.96180105), (Document(metadata={'source': '/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향.pdf', 'page': 13}, page_content='전략'), 1.0732026), (Document(metadata={'source': '/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향.pdf', 'page': 17}, page_content='이노비즈 정책브리프\n발행인 |임병훈\n편집인 |김세종\n발행일 |2021년 5월 18일\n발행처 |이노비즈정책연구원\n경기도 성남시 분당구 판교로 255(삼평동 ) 이노밸리 E동 202호\n전화 : 031-628-9600   팩스 : 031-628-9611  

In [ ]:
query = '탄소중립사회'
results = db.similarity_search_with_relevance_scores(query, k=3)

print('질문:' + query)

for i in range(len(results)):
  print('{0}번째 유사도 문서 \n{1}'.format(i+1, round(results[i][1],2)))
  print('-' * 100)
  print(results[i][0].page_content)
  print('-' * 100)
  print(results[i][0].metadata)

질문:탄소중립사회
1번째 유사도 문서 
0.66
----------------------------------------------------------------------------------------------------
01이노비즈 정책브리프
01 탄소중립과 대응 필요성
탄소중립이란 ?
∙탄소중립은 개인·회사·단체 등에서 배출되는 탄소와 흡수되는 탄소량을 같게 해 탄소 ‘순배출이 제
로(0)’가 되게 하는 것으로 , ‘넷-제로(Net-Zero)’ 라 부르기도 함
- 인간의 활동에 의한 온실가스 * 배출을 최대한 줄이고 , 남은 온실가스는 흡수, 제거하여 ‘순배출이 제로
(0)’가 되는 개념이며 , 기후변화에 대응하기 위한 글로벌 추세로 자리 잡고 있음
*온실가스란 ‘지구 대기를 오염시켜 온실 효과를 일으키는 가스’를 통틀어 이르는 말로, 이산화탄소 (CO2), 메탄(CH4), 아산화질소 (N2O), 수
소불화탄소 (HFCs), 과불화탄소 (PFCs), 육불화황 (SF6) 등은 배출 규제 및 거래의 대상이 되고 있음 
대응 필요성
∙지구 온난화로 인한 폭염, 폭설, 태풍, 산불 등 이상기후 현상이 세계 곳곳에서 발생되고 있는 가
운데 기후변화 문제에 대한 국제사회의 공동 대응 필요성이 대두됨
----------------------------------------------------------------------------------------------------
{'source': '/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향.pdf', 'page': 3}
2번째 유사도 문서 
0.64
----------------------------------------------------------------------------------------------------
보제공 확대 
- 재생원료 사용 극대화 , 철강, 플라스틱을 대체하는 혁신소재 개발, 탄소발자국 * 등 친환경 제품 정보 제
공 확대 등 순환 사